### Тема “Рекуррентные блоки”

На вебинаре мы говорили что долгое время CNN и RNN архитектуры были конкурирующими, выяснить какая архитектура больше подходит для задачи сентимент анализа на данных с вебинара

1. построить свёрточную архитектуру
2. построить различные архитектуры с RNN
3. построить совместные архитектуры CNN -> RNN и/или (RNN -> CNN)
4. сделать выводы что получилось лучше

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
import tensorflow_addons as tfa

from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.losses import SparseCategoricalCrossentropy
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, SimpleRNN, LSTM, GRU, Masking
from keras.callbacks import TensorBoard 
from keras.callbacks import EarlyStopping 

In [3]:
df =  pd.read_excel('отзывы за лето.xls')

In [4]:
df.head()

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


In [5]:
df_train, df_test = train_test_split(df, test_size=0.33, random_state=42)

In [7]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

In [8]:
df_train['Content'] = df_train['Content'].apply(preprocess_text)
df_test['Content'] = df_test['Content'].apply(preprocess_text)

In [9]:
text_corpus_train = df_train['Content'].values
text_corpus_test = df_test['Content'].values

tokenizer = Tokenizer(num_words=None, 
                     filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n', #символы для удаления из текстов
                     lower = False, #перевод к нижнему регистру
                      split = ' ') #разделитель для слов

tokenizer.fit_on_texts(text_corpus_train)

len(tokenizer.index_word)

10174

In [10]:
sequences_train = tokenizer.texts_to_sequences(text_corpus_train)
sequences_test = tokenizer.texts_to_sequences(text_corpus_test)

sequences_train[:10]

[[593, 39, 756, 3488, 20, 1, 2, 14],
 [2, 170],
 [5],
 [57],
 [2],
 [120, 1],
 [35, 949, 64, 97, 28, 466, 868],
 [95, 48, 1, 669, 129, 713, 99, 1931, 50, 1932, 1599, 2],
 [194, 137, 92, 19, 104, 1037, 48, 9],
 [24, 1345, 869, 5, 3489, 714]]

In [11]:
word_count = len(tokenizer.index_word) + 1 
training_length = max([len(i.split()) for i in text_corpus_train])
word_count, training_length 

(10175, 108)

In [12]:
X_train = pad_sequences(sequences_train, maxlen=training_length)
X_test = pad_sequences(sequences_test, maxlen=training_length)

In [13]:
le = LabelEncoder()
train_enc_labels = le.fit_transform(df_train['Rating']) 
test_enc_labels = le.transform(df_test['Rating'])
le.classes_

num_classes = 5
y_train = tf.keras.utils.to_categorical(train_enc_labels, num_classes=num_classes)
y_test = tf.keras.utils.to_categorical(test_enc_labels, num_classes=num_classes)

#### CONV модель

In [14]:
model = Sequential()
model.add(Embedding(input_dim=word_count, output_dim=128, input_length=training_length))                   
model.add(Conv1D(128, 3))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Dense(5))
model.add(Activation('softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 108, 128)          1302400   
                                                                 
 conv1d (Conv1D)             (None, 106, 128)          49280     
                                                                 
 global_max_pooling1d (Globa  (None, 128)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 10)                1290      
                                                                 
 dense_1 (Dense)             (None, 5)                 55        
                                                                 
 activation (Activation)     (None, 5)                 0         
                                                        

2022-11-14 16:27:22.750927: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [15]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', tfa.metrics.F1Score(num_classes=num_classes)])

In [16]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True) 

epochs = 20
batch_size = 512

In [16]:
%time
history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Epoch 1/20
25/25 [==============================] - 4s 115ms/step - loss: 1.0555 - accuracy: 0.6750 - f1_score: 0.1681 - val_loss: 0.8463 - val_accuracy: 0.7011 - val_f1_score: 0.1649
Epoch 2/20
25/25 [==============================] - 3s 113ms/step - loss: 0.7399 - accuracy: 0.7395 - f1_score: 0.2614 - val_loss: 0.7246 - val_accuracy: 0.7386 - val_f1_score: 0.2732
Epoch 3/20
25/25 [==============================] - 3s 111ms/step - loss: 0.6270 - accuracy: 0.7776 - f1_score: 0.3149 - val_loss: 0.6965 - val_accuracy: 0.7480 - val_f1_score: 0.2984
Epoch 4/20
25/25 [==============================] - 3s 108ms/step - loss: 0.5505 - accuracy: 0.8044 - f1_score: 0.3757 - val_loss: 0.6858 - val_accuracy: 0.7574 - val_f1_score: 0.3366
Epoch 5/20
25/25 [==============================] - 3s 108ms/step - loss: 0.4821 - accuracy: 0.8310 - f1_score: 0.4389 - val_loss: 0.6938 - val_accuracy: 0.7581 - val_f1_score: 0.3457
Epoch 6/20
25/25 [==============================] - 3s 108ms/step - loss: 0.4173

In [17]:
score_conv = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

14/14 [==============================] - 0s 22ms/step - loss: 1.2696 - accuracy: 0.7375 - f1_score: 0.3735


#### SimpleRNN

In [18]:
model = Sequential()

model.add(Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(SimpleRNN(64))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(5, activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 108, 30)           305250    
                                                                 
 masking (Masking)           (None, 108, 30)           0         
                                                                 
 simple_rnn (SimpleRNN)      (None, 64)                6080      
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_3 (Dense)             (None, 5)                 325       
                                                                 
Total params: 315,815
Trainable params: 315,815
Non-tr

In [19]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', tfa.metrics.F1Score(num_classes=num_classes)])

In [20]:
%time
history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Epoch 1/20
25/25 [==============================] - 4s 118ms/step - loss: 1.2159 - accuracy: 0.6396 - f1_score: 0.1750 - val_loss: 1.0102 - val_accuracy: 0.7011 - val_f1_score: 0.1649
Epoch 2/20
25/25 [==============================] - 3s 107ms/step - loss: 0.9441 - accuracy: 0.7038 - f1_score: 0.1855 - val_loss: 0.8640 - val_accuracy: 0.7011 - val_f1_score: 0.2034
Epoch 3/20
25/25 [==============================] - 3s 105ms/step - loss: 0.8188 - accuracy: 0.7168 - f1_score: 0.2620 - val_loss: 0.7822 - val_accuracy: 0.7249 - val_f1_score: 0.2566
Epoch 4/20
25/25 [==============================] - 3s 102ms/step - loss: 0.7237 - accuracy: 0.7500 - f1_score: 0.2926 - val_loss: 0.7341 - val_accuracy: 0.7357 - val_f1_score: 0.2692
Epoch 5/20
25/25 [==============================] - 3s 107ms/step - loss: 0.6498 - accuracy: 0.7773 - f1_score: 0.3199 - val_loss: 0.7128 - val_accuracy: 0.7415 - val_f1_score: 0.2883
Epoch 6/20
25/25 [==============================] - 3s 104ms/step - loss: 0.6528

In [21]:
score_srnn= model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

14/14 [==============================] - 0s 34ms/step - loss: 1.0677 - accuracy: 0.7392 - f1_score: 0.3668


#### LSTM

In [22]:
model = Sequential()

model.add(Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(LSTM(64, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(5, activation='softmax'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 108, 30)           305250    
                                                                 
 masking_1 (Masking)         (None, 108, 30)           0         
                                                                 
 lstm (LSTM)                 (None, 64)                24320     
                                                                 
 dense_4 (Dense)             (None, 64)                4160      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_5 (Dense)             (None, 5)                 325       
                                                                 
Total params: 334,055
Trainable params: 334,055
Non-tr

In [23]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', tfa.metrics.F1Score(num_classes=num_classes)])

In [24]:
%time
history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Epoch 1/20
25/25 [==============================] - 11s 324ms/step - loss: 1.2236 - accuracy: 0.6924 - f1_score: 0.1704 - val_loss: 0.9689 - val_accuracy: 0.7011 - val_f1_score: 0.1649
Epoch 2/20
25/25 [==============================] - 8s 302ms/step - loss: 0.9052 - accuracy: 0.7030 - f1_score: 0.1652 - val_loss: 0.8351 - val_accuracy: 0.7040 - val_f1_score: 0.1766
Epoch 3/20
25/25 [==============================] - 8s 300ms/step - loss: 0.7836 - accuracy: 0.7326 - f1_score: 0.2480 - val_loss: 0.7536 - val_accuracy: 0.7314 - val_f1_score: 0.2625
Epoch 4/20
25/25 [==============================] - 8s 302ms/step - loss: 0.7093 - accuracy: 0.7608 - f1_score: 0.2857 - val_loss: 0.7215 - val_accuracy: 0.7372 - val_f1_score: 0.2703
Epoch 5/20
25/25 [==============================] - 7s 300ms/step - loss: 0.6538 - accuracy: 0.7726 - f1_score: 0.2967 - val_loss: 0.7056 - val_accuracy: 0.7444 - val_f1_score: 0.2775
Epoch 6/20
25/25 [==============================] - 8s 301ms/step - loss: 0.608

In [25]:
score_lstm = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

14/14 [==============================] - 1s 78ms/step - loss: 1.1256 - accuracy: 0.7394 - f1_score: 0.3898


#### GRU

In [26]:
model = Sequential()

model.add(Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(GRU(64,recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(5, activation='softmax'))
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 108, 30)           305250    
                                                                 
 masking_2 (Masking)         (None, 108, 30)           0         
                                                                 
 gru (GRU)                   (None, 64)                18432     
                                                                 
 dense_6 (Dense)             (None, 64)                4160      
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense_7 (Dense)             (None, 5)                 325       
                                                                 
Total params: 328,167
Trainable params: 328,167
Non-tr

In [27]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', tfa.metrics.F1Score(num_classes=num_classes)])

In [28]:
%time
history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Epoch 1/20
25/25 [==============================] - 10s 279ms/step - loss: 1.2823 - accuracy: 0.6918 - f1_score: 0.1765 - val_loss: 0.9330 - val_accuracy: 0.7011 - val_f1_score: 0.1649
Epoch 2/20
25/25 [==============================] - 6s 260ms/step - loss: 0.8540 - accuracy: 0.7030 - f1_score: 0.1651 - val_loss: 0.7990 - val_accuracy: 0.7011 - val_f1_score: 0.1649
Epoch 3/20
25/25 [==============================] - 7s 261ms/step - loss: 0.7442 - accuracy: 0.7241 - f1_score: 0.2300 - val_loss: 0.7439 - val_accuracy: 0.7162 - val_f1_score: 0.2357
Epoch 4/20
25/25 [==============================] - 7s 264ms/step - loss: 0.6777 - accuracy: 0.7652 - f1_score: 0.2881 - val_loss: 0.7260 - val_accuracy: 0.7336 - val_f1_score: 0.2647
Epoch 5/20
25/25 [==============================] - 6s 254ms/step - loss: 0.6181 - accuracy: 0.7802 - f1_score: 0.3021 - val_loss: 0.7214 - val_accuracy: 0.7430 - val_f1_score: 0.2775
Epoch 6/20
25/25 [==============================] - 6s 242ms/step - loss: 0.571

In [29]:
score_gru= model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

14/14 [==============================] - 1s 59ms/step - loss: 1.3509 - accuracy: 0.7357 - f1_score: 0.3845


In [30]:
result_RNN = pd.DataFrame({'model': ['SimpleRNN', 'LSTM', 'GRU'], 'f1_score': [score_srnn[1], score_lstm[1], score_gru[1]]})
result_RNN.sort_values(by='f1_score', ascending=False)

,model,f1_score
1,LSTM,0.739366
0,SimpleRNN,0.739220
2,GRU,0.735700


Лидером по качеству является LSTM, но с учетом того, что обучение происходит значительно дольше, чем Simple RNN, при незначительно большей точности - в большинстве ситуаций целесообразнее будет использовать именно Simple RNN.

#### CNN -> RNN

In [31]:
model = Sequential()

model.add(Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(LSTM(64,recurrent_dropout=0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(5, activation='softmax'))
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 108, 30)           305250    
                                                                 
 masking_3 (Masking)         (None, 108, 30)           0         
                                                                 
 conv1d_1 (Conv1D)           (None, 106, 128)          11648     
                                                                 
 activation_1 (Activation)   (None, 106, 128)          0         
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 dense_8 (Dense)             (None, 32)                2080      
                                                                 
 dropout_3 (Dropout)         (None, 32)               

In [32]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', tfa.metrics.F1Score(num_classes=num_classes)])

In [33]:
%time
history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

CPU times: user 2 µs, sys: 2 µs, total: 4 µs
Wall time: 7.87 µs
Epoch 1/20
25/25 [==============================] - 14s 438ms/step - loss: 1.2374 - accuracy: 0.6776 - f1_score: 0.1802 - val_loss: 0.9344 - val_accuracy: 0.7011 - val_f1_score: 0.1649
Epoch 2/20
25/25 [==============================] - 10s 415ms/step - loss: 0.8807 - accuracy: 0.7072 - f1_score: 0.1908 - val_loss: 0.7906 - val_accuracy: 0.7148 - val_f1_score: 0.2498
Epoch 3/20
25/25 [==============================] - 10s 419ms/step - loss: 0.7506 - accuracy: 0.7490 - f1_score: 0.2884 - val_loss: 0.7398 - val_accuracy: 0.7350 - val_f1_score: 0.2701
Epoch 4/20
25/25 [==============================] - 10s 410ms/step - loss: 0.6612 - accuracy: 0.7718 - f1_score: 0.3005 - val_loss: 0.7194 - val_accuracy: 0.7451 - val_f1_score: 0.2894
Epoch 5/20
25/25 [==============================] - 10s 413ms/step - loss: 0.5982 - accuracy: 0.7890 - f1_score: 0.3332 - val_loss: 0.7288 - val_accuracy: 0.7451 - val_f1_score: 0.3140
Epoch 6/20


In [34]:
score_cnnrnn= model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

14/14 [==============================] - 2s 108ms/step - loss: 1.3003 - accuracy: 0.7290 - f1_score: 0.3806


#### RNN -> CNN

In [35]:
model = Sequential()

model.add(Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(LSTM(64,recurrent_dropout=0.2, return_sequences=True))
model.add(Conv1D(64, 3))
model.add(GlobalMaxPool1D())
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(5, activation='softmax'))
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 108, 30)           305250    
                                                                 
 masking_4 (Masking)         (None, 108, 30)           0         
                                                                 
 lstm_2 (LSTM)               (None, 108, 64)           24320     
                                                                 
 conv1d_2 (Conv1D)           (None, 106, 64)           12352     
                                                                 
 global_max_pooling1d_1 (Glo  (None, 64)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_10 (Dense)            (None, 32)                2080      
                                                      

In [36]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', tfa.metrics.F1Score(num_classes=num_classes)])

In [37]:
%time
history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

CPU times: user 2 µs, sys: 2 µs, total: 4 µs
Wall time: 8.11 µs
Epoch 1/20
25/25 [==============================] - 12s 368ms/step - loss: 1.2722 - accuracy: 0.6757 - f1_score: 0.1759 - val_loss: 1.0558 - val_accuracy: 0.7011 - val_f1_score: 0.1649
Epoch 2/20
25/25 [==============================] - 9s 354ms/step - loss: 1.0302 - accuracy: 0.7019 - f1_score: 0.1656 - val_loss: 1.0030 - val_accuracy: 0.7011 - val_f1_score: 0.1649
Epoch 3/20
25/25 [==============================] - 9s 351ms/step - loss: 1.0234 - accuracy: 0.7024 - f1_score: 0.1650 - val_loss: 1.0022 - val_accuracy: 0.7011 - val_f1_score: 0.1649
Epoch 4/20
25/25 [==============================] - 9s 343ms/step - loss: 1.0119 - accuracy: 0.7025 - f1_score: 0.1651 - val_loss: 1.0014 - val_accuracy: 0.7011 - val_f1_score: 0.1649
Epoch 5/20
25/25 [==============================] - 9s 346ms/step - loss: 1.0125 - accuracy: 0.7030 - f1_score: 0.1651 - val_loss: 1.0044 - val_accuracy: 0.7011 - val_f1_score: 0.1649
Epoch 6/20
25/2

In [38]:
score_rnncnn= model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)

14/14 [==============================] - 1s 91ms/step - loss: 0.8999 - accuracy: 0.7432 - f1_score: 0.3743


In [39]:
result_CNN_RNN = pd.DataFrame({'model': ['CNN_RNN', 'RNN_CNN'], 'f1_score': [score_cnnrnn[1], score_rnncnn[1]]})
result_CNN_RNN.sort_values(by='f1_score', ascending=False)

,model,f1_score
1,RNN_CNN,0.743180
0,CNN_RNN,0.728953


RNN_CNN показывает лучшую точность

Общий итог

In [40]:
result_CNN = pd.DataFrame({'model': 'CNN', 'f1_score': [score_conv[1]]})
result_models = pd.concat([result_CNN, result_RNN, result_CNN_RNN], ignore_index=True)
result_models.sort_values(by='f1_score', ascending=False)

,model,f1_score
5,RNN_CNN,0.743180
2,LSTM,0.739366
1,SimpleRNN,0.739220
0,CNN,0.737460
3,GRU,0.735700
4,CNN_RNN,0.728953
